In [83]:
from Bio.PDB.Polypeptide import three_to_one
from Bio.SVDSuperimposer import SVDSuperimposer
from Bio.PDB.Structure import Structure
from Bio.PDB import PDBParser, MMCIFParser
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import numpy as np


AA = ["ALA", "CYS", "ASP", "GLU", "PHE", "GLY", "HIS", "ILE", "LYS", "LEU", "MET", "ASN", "PRO", "GLN",
      "ARG", "SER", "THR", "VAL", "TRP", "TYR"]

parser = PDBParser(QUIET=True)
mmcif_parser = MMCIFParser(QUIET=True)
    
native = parser.get_structure("native", "af3_predictions/fold_t1124_new/7ux8.pdb")
model = mmcif_parser.get_structure("model", "af3_predictions/fold_t1124_new/fold_t1124_new_model_0.cif")

# Type functions because it's helpful.
def align(native: Structure, model: Structure, atom_types = ["CA", "N", "C", "O"]) -> SVDSuperimposer:
    """
    Aligns a model structure onto a native structure
    Using the atom types listed in `atom_types`.
    """
    
    # A long one-liner that gets the one-letter amino acid representation for each residue in a structure,
    # then joins those letters into one long string.
    native_seq = "".join([ three_to_one(r.resname) for r in native[0].get_residues() if r.resname in AA ])
    model_seq = "".join([ three_to_one(r.resname) for r in model[0].get_residues() if r.resname in AA ])

    #print(len(native_seq))
    #print(len(model_seq))

    alignments = pairwise2.align.globalxx(native_seq, model_seq)

    best_alignment = alignments[0]
    #print(best_alignment)

    aligned_native = best_alignment[0]
    aligned_model = best_alignment[1]
    start, end = best_alignment[3:7]

    # Determine which sequence to trim
    if len(native_seq) > len(model_seq):
        # Trim seq1 to match the length of seq2 based on alignment
        trimmed_native = native_seq[start:end]
        trimmed_model = model_seq
    else:
        # Trim seq2 to match the length of seq1 based on alignment
        trimmed_native = native_seq
        trimmed_model = model_seq[start:end]
    
    
    # # Get the coordinates of the Atom object if the Atom is from an amino acid residue,
    # # and the atom type is what's specified in atom_types.
    # # Traditionally RMSD is calculated for either:
    # # Only the alpha-carbon atoms (CA), or
    # # The "protein backbone" atoms (CA, N, C, O), or
    # # All atoms
    # native_coords = [ a.coord for a in native[0].get_atoms() if a.parent.resname in AA and a.name in atom_types ]
    # model_coords = [ a.coord for a in model[0].get_atoms() if a.parent.resname in AA and a.name in atom_types ]
    
    # si = SVDSuperimposer()
    # si.set(np.array(native_coords), np.array(model_coords))
    # si.run() # Run the SVD alignment
    
    # return si

In [84]:
#si = align(native, model)
#print("Initial RMSD: {:.2f} angstroms; full-backbone RMSD after alignment: {:.2f} angstroms".format(si.get_init_rms(), si.get_rms()))

AttributeError: 'NoneType' object has no attribute 'get_init_rms'